In [1]:
import os
import torch
import pytorch_lightning as L
import matplotlib.pyplot as plt
from omegaconf import DictConfig

%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

# Get number of cores
print(os.cpu_count())

# Get number of threads
print(torch.get_num_threads())

True
NVIDIA GeForce GTX 1050
3167551488
12
6


In [2]:
from pytorch_lightning.loggers import WandbLogger

from src import RoadDataModule, RoadModel, LogPredictionsCallback, val_checkpoint, regular_checkpoint


def main(cfg: DictConfig) -> None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RoadModel(cfg, device)
    datamodule = RoadDataModule(cfg)

    wandb_logger = WandbLogger(project="road-segmentation", name="test")

    trainer = L.Trainer(max_epochs=5, accelerator="gpu", devices=1, logger=wandb_logger, callbacks=[
        LogPredictionsCallback(),
        val_checkpoint,
        regular_checkpoint
    ])
    # trainer.fit(model, datamodule=datamodule)
    trainer.test(model, datamodule=datamodule)
    # trainer.predict(model, datamodule=datamodule)

In [4]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="conf"):
    # Make sure that the cfg.ds is 'robotour' 
    config = compose(config_name="config", overrides=["ds=robotour"])
    print(config)
    # main(config)

{'action': None, 'ckpt_path': 'checkpoints/e51-iou0.60.ckpt', 'run_name': 'baseline', 'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 0.0001, 'weight_decay': 0.0001}, 'train': {'batch_size': 4, 'max_epochs': 100, 'num_workers': 4}, 'ds': {'name': 'rugd', 'path': 'data/RoboTour', 'mean': [0.404, 0.404, 0.4016], 'std': [0.2646, 0.2668, 0.2685], 'color_map': {'0,0,0': 0, '0,255,0': 1, '255,0,0': 2, '0,0,255': 3}}, 'model': {'_target_': 'torchvision.models.segmentation.deeplabv3_resnet50', 'num_classes': 4}}
